In [1]:
import pandas as pd
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Задача

Задача, которую вам предстоит решать, была поставлена на [соревновании](https://russe.nlpub.org/2018/wsi/) в рамках конференции Dialog-2018.

---

### Краткое описание

Ваша задача разработать систему, способную разрешать неоднозначность, возникающую в употреблении омонимичных форм.

### Развернутое описание

In [2]:
data = pd.read_csv("/home/pavel/MyDocs/Machine Learning/Tinkoff/lecture2/train.csv", sep='\t')
data.head()

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,замок,1,NaN,NaN,замок владимира мономаха в любече . многочисле...
1,2,замок,1,NaN,NaN,"шильонский замок замок шильйон ( ) , известный..."
2,3,замок,1,NaN,NaN,проведения архитектурно - археологических рабо...
3,4,замок,1,NaN,NaN,"топи с . , л . белокуров легенда о завещании м..."
4,5,замок,1,NaN,NaN,великий князь литовский гедимин после успешной...


In [3]:
print(data.loc[30, 'context'])

, уже в стародавние времена действовала переправа через реку , а укрепление пруссов на твангсте так и напрашивалось на закладку орденской крепости . замок был выстроен из дерева на месте прусского городища тувангсте и на протяжении всего xiii века неоднократно подвергался нападениям восставших пруссов и литовских отрядов . замок был основан в январе года как деревянное строение , двумя годами позже началось возведение кирпичного замка . замок служил резиденцией маршалов тевтонского ордена и был центром сбора рыцарских походов в великое княжество литовское в xiv веке . с


In [4]:
k = data['context'].tolist()
print(k[0])

замок владимира мономаха в любече . многочисленные укрепленные монастыри также не являлись замками как таковыми — это были крепости . ранние европейские замки строились преимущественно из дерева они опоясывались деревянной оградой — палисадом уже тогда вокруг замков стали появляться рвы . примером такого замка может служить вышгородский замок киевских князей . каменное замковое строительство распространилось в западной и центральной европе лишь к xii веку . главной частью средневекового замка являлась центральная башня — донжон , выполнявшая функции цитадели . помимо своих оборонительных функций , донжон являлся непосредственным жилищем феодала . также в главной башне


Вам дается набор "главных слов". Или слов, которые имеют несколько возможных смыслов в тексте.

Например, слово `лук` может встречаться в значении `оружие` или в значении `овощ`.

Ваша задача сопоставить одинаковые метки тем контекстам, в которых "главное слово" встречается в одном и том же значении.
Важно учесть, что предполагается, что число возможных смыслов у "главного слова" заранее неизвестно. Таким образом это фактически задача кластеризации с заранее неизвестным числом классов.

Также предполагается, что система будет применяться к "главным словам", которых нет в вашей обучающей выборке.

В текущем варианте задания ваша задача построить и протестировать систему на датасете `wiki-wiki`, собранном из статей википедии.

Это очень маленький датасет, в нем всего 4 "главных слова":

In [5]:
data.word.unique()

array(['замок', 'лук', 'суда', 'бор'], dtype=object)

Т.к. это тренировочный датасет, правильные метки для каждого контекста известны и можно посмотреть некоторую статистику по ним.

**Сколько контекстов приходится на каждое слово?**

In [6]:
data.loc[:,['word', 'gold_sense_id', 'context']]\
.groupby(['word']).count()['context'].sort_values(ascending=False)

word
замок    138
суда     135
лук      110
бор       56
Name: context, dtype: int64

**Cколько контекстов приходится на одно смысловое значение слова?**

In [7]:
data.loc[:,['word', 'gold_sense_id', 'context']]\
.groupby(['word', 'gold_sense_id'])['context'].count()

word   gold_sense_id
бор    1                 14
       2                 42
замок  1                100
       2                 38
лук    1                 65
       2                 45
суда   1                100
       2                 35
Name: context, dtype: int64

In [8]:
data.loc[:,['word', 'gold_sense_id', 'context']]\
.groupby(['word', 'gold_sense_id'])['context'].count()

word   gold_sense_id
бор    1                 14
       2                 42
замок  1                100
       2                 38
лук    1                 65
       2                 45
суда   1                100
       2                 35
Name: context, dtype: int64

**Важное замечание**

Как можно заметить, в этом датасете ровно по 2 смысла у каждого слова.

Таким образом, разрешается в качестве `k` у `KMeans` взять значение 2.

```python

from sklearn.cluster import KMeans

KMeans(
```

**Про кластеризацию**


In [9]:
from sklearn.cluster import KMeans

In [10]:
# запускаем кластеризацию отдельно для каждого набора контекстов, соответствующих своему "главному слову"
km = KMeans(n_clusters=2)
# Таким образом запускаем
# km.fit_predict(X)


**Что нужно сделать, чтобы посчитать метрику ARI?**

```python
from sklearn.metrics import adjusted_rand_score
```
Передать туда предсказанные метки и `gold_sense_id` из датасета.




---
**План**
1. Ознакомится с постановкой задачи и метрикой [ARI](https://en.wikipedia.org/wiki/Rand_index#Adjusted_Rand_index)
2. Получить вектора-представления слов. Возможные пути решения:
    - Обучить эмбеддинги с помощью [FastText](https://github.com/facebookresearch/fastText) на найденном вами же большом корпусе русского языка. 
    - Скачать готовые эмбеддинги. Это потребует определенного препроцессинга тестового датасета, чтобы слова в нем соответствовали словам в словаре скачанных эмбеддингов (например, в случае использования [rusVectores](http://rusvectores.org/ru/models/) потребуется добавить к каждому слову часть речи).
3. Придумать способ представления контекста с помощью имеющихся эмбеддингов слов. Hint: можно также воспользоваться знаниями о TF-IDF представлении текста в совокупности с предобученными эмбеддингами слов.
4. Воспользоваться алгоритмом кластеризации KMeans чтобы собрать контексты с одинаковым смысловым значением "главного слова" в один кластер. 
5. Посчитать метрику ARI для полученной кластеризации.

Лучшее представление текста дает лучшую кластеризацию. Фокус данного задания не на алгоритме кластеризации, а на получении наиболее хорошего представления для векторов контекста.

---
Для удобства работы с эмбеддингами предлагается воспользоваться пакетом `gensim`

# Homework 2

### Grouping data

In [11]:
dataGroups = []
for word in data.word.unique():
    dataGroups.append(data.loc[data['word'] == word])

## Text preprocessing

In [12]:
from __future__ import print_function
from __future__ import division
from future import standard_library
import sys
import os
import wget
from ufal.udpipe import Model, Pipeline

In [13]:
def tag_ud(pipeline, text='Текст нужно передать функции в виде строки!', pos=True):
    # если частеречные тэги не нужны (например, их нет в модели), выставьте pos=False
    # в этом случае на выход будут поданы только леммы

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста лемму и тэг
    tagged = [w.split('\t')[2].lower() + '_' + w.split('\t')[3] for w in content if w]

    tagged_propn = []
    propn = []
    for t in tagged:
        if t.endswith('PROPN'):
            if propn:
                propn.append(t)
            else:
                propn = [t]
        elif t.endswith('PUNCT'):
            propn = []
            continue  # я здесь пропускаю знаки препинания, но вы можете поступить по-другому
        else:
            if len(propn) > 1:
                name = '::'.join([x.split('_')[0] for x in propn]) + '_PROPN'
                tagged_propn.append(name)
            elif len(propn) == 1:
                tagged_propn.append(propn[0])
            tagged_propn.append(t)
            propn = []
    if not pos:
        tagged_propn = [t.split('_')[0] for t in tagged_propn]
    return tagged_propn

## Word embaddings

In [14]:
from gensim.models import KeyedVectors
wordVec = KeyedVectors.load_word2vec_format("/home/pavel/MyDocs/Machine Learning/Tinkoff/lecture2/ruscorpora_upos_skipgram_300_5_2018.vec", binary=False)

## First approach - Average of Word2Vec vectors

In [15]:
model = Model.load('udpipe_syntagrus.model')
pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

In [16]:
def text_vector_average(data, wordVec):
    textVec= []
    first = int(data['context_id'].tolist()[0]) - 1
    for i in range(data.shape[0]):
        words = tag_ud(pipeline, data.loc[first + i, 'context'])
        vectors = []
        for word in words:
            if word in wordVec:
                vectors.append(wordVec[word])
        textVec.append(np.average(vectors, axis=0)) 
    return textVec

In [17]:
from sklearn.metrics import adjusted_rand_score

def predict_precesion_avr(num, dataGroups, wordVec):
    textVec = text_vector_average(dataGroups[num], wordVec)
    km = KMeans(n_clusters=2)
    pred_sense = km.fit_predict(textVec)
    gold_sense = dataGroups[num]['gold_sense_id'].values
    res = adjusted_rand_score(pred_sense, gold_sense)
    return res

In [18]:
res1 = predict_precesion_avr(0, dataGroups, wordVec)
print(res1)

0.22418165790824132


In [19]:
res2 = predict_precesion_avr(1, dataGroups, wordVec)
print(res2)

0.9279187206925574


In [20]:
res3 = predict_precesion_avr(2, dataGroups, wordVec)
print(res3)

0.32101725080207605


In [21]:
res4 = predict_precesion_avr(3, dataGroups, wordVec)
print(res4)

1.0


### Среднее значение ARI

In [22]:
print((res1 + res2 + res3 + res4)/4)

0.6182794073507187


# Second approach - Average of Word2Vec vectors with TF-IDF without vectors normalization

In [23]:
import math
from textblob import TextBlob as tb

def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.words)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [24]:
def tf_idf_avrg_unnorm(contexts, words, vectors, textNum):
    newVectors = []
    for i in range(len(words)):
        word = words[i]
        vec = vectors[i]
        tfIdfScore = tfidf(word, contexts[textNum], contexts)
        newVectors.append(tfIdfScore*vec)
    return np.average(newVectors, axis=0)

In [25]:
def text_vector_tfidf_unnorm(contexts, data, wordVec):
    textVec= []
    first = int(data['context_id'].tolist()[0]) - 1
    for i in range(data.shape[0]):
        #print("Text", i, "is OK")
        words = contexts[i].split()
        vectors = []
        validWords = []
        for word in words:
            if word in wordVec:
                vectors.append(wordVec[word])
                validWords.append(word)
        textVec.append(tf_idf_avrg_unnorm(contexts, validWords, vectors, i)) 
    return textVec

In [26]:
def set_contexts(n):
    contexts = []
    first = int(dataGroups[n]['context_id'].tolist()[0]) - 1
    for i in range(dataGroups[n].shape[0]):
        text = ' '.join(tag_ud(pipeline, dataGroups[n].loc[first + i, 'context']))
        contexts.append(tb(text))
    return contexts

In [27]:
def predict_precesion_tfidf_unnorm(num, dataGroups, wordVec):
    contexts = set_contexts(num)
    textVec = text_vector_tfidf_unnorm(contexts, dataGroups[num], wordVec)
    km = KMeans(n_clusters=2)
    pred_sense = km.fit_predict(textVec)
    gold_sense = dataGroups[num]['gold_sense_id'].values
    res = adjusted_rand_score(pred_sense, gold_sense)
    return res

In [28]:
res1 = predict_precesion_tfidf_unnorm(0, dataGroups, wordVec)
print(res1)

/home/pavel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  


0.02348013466993366


In [29]:
res2 = predict_precesion_tfidf_unnorm(1, dataGroups, wordVec)
print(res2)

/home/pavel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  


0.08547435755153875


In [30]:
res3 = predict_precesion_tfidf_unnorm(2, dataGroups, wordVec)
print(res3)

/home/pavel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  


0.4247564197996499


In [31]:
res4 = predict_precesion_tfidf_unnorm(3, dataGroups, wordVec)
print(res4)

/home/pavel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  


0.051325199831720625


## Среднее значение ARI

In [32]:
print((res1+res2+res3+res4)/4)

0.14625902796321072


# Third approach - Average of Word2Vec vectors with TF-IDF with vectors normalization

In [33]:
from sklearn.preprocessing import normalize

In [34]:
def tf_idf_avrg(contexts, words, vectors, textNum):
    newVectors = []
    for i in range(len(words)):
        word = words[i]
        vec = vectors[i]
        tfIdfScore = tfidf(word, contexts[textNum], contexts)
        newVectors.append(tfIdfScore*vec)
    res = np.average(newVectors, axis=0)
    if(np.linalg.norm(res) > 0):
        res = res/np.linalg.norm(res)
    return res

In [35]:
def text_vector_tfidf(contexts, data, wordVec):
    textVec= []
    first = int(data['context_id'].tolist()[0]) - 1
    for i in range(data.shape[0]):
        #print("Text", i, "is OK")
        words = contexts[i].split()
        vectors = []
        validWords = []
        for word in words:
            if word in wordVec:
                vectors.append(wordVec[word])
                validWords.append(word)
        textVec.append(tf_idf_avrg(contexts, validWords, vectors, i)) 
    return textVec

In [36]:
def predict_precesion_tfidf(num, dataGroups, wordVec):
    contexts = set_contexts(num)
    textVec = text_vector_tfidf(contexts, dataGroups[num], wordVec)
    km = KMeans(n_clusters=2)
    pred_sense = km.fit_predict(textVec)
    gold_sense = dataGroups[num]['gold_sense_id'].values
    res = adjusted_rand_score(pred_sense, gold_sense)
    return res

In [37]:
res1 = predict_precesion_tfidf(0, dataGroups, wordVec)
print(res1)

/home/pavel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  


0.7939421411639748


In [38]:
res2 = predict_precesion_tfidf(1, dataGroups, wordVec)
print(res2)

/home/pavel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  


0.9279187206925574


In [39]:
res3 = predict_precesion_tfidf(2, dataGroups, wordVec)
print(res3)

/home/pavel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  


0.33798134638470767


In [40]:
res4 = predict_precesion_tfidf(3, dataGroups, wordVec)
print(res4)

/home/pavel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  


1.0


## Среднее значение ARI

In [41]:
print((res1+res2+res3+res4)/4)

0.76496055206031


## Отчёт:

Я также пробовал обучать tf-idf на всем корпусе текстов из train.csv, а не отдельно для каждого из главных слов, но это давало очень маленькие значения метрики ARI (среднее значение было отрицательным). Также пробовал использовать не tf-idf, а idf, в этом случае значение метрики ARI было 0.637 (0.28, 0.93, 0.34, 1), что почти тоже самое, что и при использовании Average of Word2Vec vectors (можно легко проверить, поменяв, в функции tf_idf_avrg "tfIdfScore = tfidf(word, contexts[textNum], contexts)" на "tfIdfScore = idf(word, contexts)") . Использование же tf давало среднее значение 0.31 (0.02, -0.01, 0.23, 1). Таким образом, луший результат дает классификация с помощью tf-idf и нормализации эмбеддингов.